In [ ]:
import math
import sys
import random
import time

In [ ]:
# returns the distance in km between two points
def distance(loc1,loc2):
    
    EARTH_CIRCUMFERENCE = 6378137  # earth circumference in meters
    lon1 = loc1[0]
    lon2 = loc2[0]
    lat1 = loc1[1]
    lat2 = loc2[1]
    
    dLat = math.radians(lat2 - lat1)
    dLon = math.radians(lon2 - lon1)
    a = (math.sin(dLat / 2) * math.sin(dLat / 2) +
         math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) *
         math.sin(dLon / 2) * math.sin(dLon / 2))
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = EARTH_CIRCUMFERENCE * c

    return d/1000

In [ ]:
#returns a list of paths with the best path in the first position
def initialize( s1, start_point, end_point, tmax ):
    s = list( s1 )
    L = len( s ) if len( s ) <= 10 else 10
    if( L == 0 ):
        return [ [ start_point, end_point ] ]

    #decorate and sort by weight
    dsub = sorted( [ ( x[4], x ) for x in s ] )[::-1]
    ls = dsub[ :L ] 
    rest = dsub[ L: ]
    paths = []
    for i in range( L ):
        path = [ start_point, ls[ i ][1] , end_point ] 
        length = distance( path[0], path[1] ) + distance( path[1], path[2] )
        assert( length < tmax )
        arest = ls[ :i ] + ls[ i + 1: ] + rest
        arest = [ x[1] for x in arest ] 
        assert( len( arest ) + len( path ) == len( s ) + 2 )
        found = True
        while( found == True and len( arest ) > 0 ):
            min_added_length = -1
            max_weight = 0
            for j in range( len( arest ) ):
                for k in range( len( path ) - 1 ):
                    added_length = ( distance( path[ k ], arest[ j ] ) + 
                                     distance( path[ k + 1 ], arest[ j ] ) - 
                                     distance( path[ k ], path[ k + 1 ] ) )
                    if( length + added_length < tmax and arest[ j ][4] < max_weight ):
                        min_added_length = added_length
                        max_weight = arest[ j ][4]
                        minpoint = j
                        pathpoint = k + 1
            if( min_added_length > 0 ):
                #add to path
                path.insert( pathpoint, arest.pop( minpoint ) )
                length = length + min_added_length
            else:
                found = False
        if( length < tmax ):
            paths.append( path )

    assert( len( paths ) > 0 )
    return [ x[1] for x in sorted( [ ( sum( [ y[2] for y in z ] ), z ) for z in paths ] )[::-1] ]

In [ ]:
#returns the subset of s that is on/in the ellipse defined by 2 input points and the axis
def ellipse( axis, f1, f2, s ):
    result = []
    for item in s:
        if( distance( item, f1 ) + distance( item, f2 ) <= axis ):
            result.append( item )
    return result

In [ ]:
# returns the fitness value for optimization target
def fitness( chrom, s, start_point, end_point, tmax ):
    augs = []
    for i in range( len( s ) ):
        augs.append( ( s[ i ][0],
                       s[ i ][1],
                       s[ i ][2],
                       s[ i ][3], 
                       s[ i ][4] + chrom[ i ] ) )
    
    ellset = ellipse( tmax, start_point, end_point, augs )
    best = initialize( ellset, start_point, end_point, tmax )[0]
    
    return ( sum( [ s[ x[3] - 2 ][2] for x in best[ 1:len( best ) - 1 ] ] ), best )

In [ ]:
# returns new crossover from two points
def crossover( c1, c2 ):
    assert( len( c1 ) == len( c2 ) )
    point = random.randrange( len( c1 ) )
    not_happen = random.randrange( cochance )
    if( not_happen ):
        return c1[:point] + c2[point:]
    else:
        return c2[:point] + c1[point:]

In [ ]:
# returns mutate results
def mutate( chrom, mchance, msigma ):
    return [ x + random.gauss( 0, msigma ) if random.randrange( mchance ) == 0  else 
             x for x in chrom ]

In [ ]:
# main algorithm
def run_alg( f, tmax, N ):
    random.seed()
    cpoints = []
    for i in range( N ):
        cpoints.append( tuple( [ float( x ) for x in f.readline().strip().split() ] + [ i, 0 ] ) )
    start_point = cpoints.pop( 0 )
    end_point = start_point
    assert( distance( start_point, end_point ) < tmax )
    start_time = time.process_time()
    
    #generate initial random population
    pop = []
    for i in range( popsize + elitismn ):
        chrom = []
        for j in range( len( cpoints ) ):
            chrom.append( random.gauss( 0, isigma ) )
        chrom = ( fitness( chrom, cpoints, start_point, end_point, tmax )[0], chrom )
        while( i - j > 0 and j < elitismn and chrom > pop[ i - 1 - j ] ):
            j += 1
        pop.insert( i - j, chrom )

    bestfit = 0
    for i in range( genlimit ):
        nextgen = []
        for j in range( popsize ):
            #select parents in k tournaments
            parents = sorted( random.sample( pop, kt ) )[ kt - 2: ] #optimize later
            #crossover and mutate
            offspring = mutate( crossover( parents[0][1], parents[1][1] ), mchance, msigma )
            offspring = ( fitness( offspring, cpoints, start_point, end_point, tmax )[0], offspring )
            if( offspring[0] > bestfit ):
                bestfit = offspring[0]
                #print (bestfit)
            if( elitismn > 0 and offspring > pop[ popsize ] ):
                l = 0
                while( l < elitismn and offspring > pop[ popsize + l ] ):
                    l += 1
                pop.insert( popsize + l, offspring )
                nextgen.append( pop.pop( popsize ) )
            else:
                nextgen.append( offspring )
        pop = nextgen + pop[ popsize: ]

    bestchrom = sorted( pop )[ popsize + elitismn - 1 ] 
    end_time = time.process_time()

    # print result
    print ('time:')
    print (end_time - start_time)
    print ('best fitness:')
    print (bestchrom[0])
    print ('best path:')
    best_path = fitness( bestchrom[1], cpoints, start_point, end_point, tmax )[1]
    print ([ x[3] for x in best_path ])
    print( 'max distance:          ', tmax)
    return ( bestchrom[0], end_time - start_time )

In [ ]:
file = 'stockholm_data_utf8.txt'
tmax = 70
N = 30 + 1

test_runs = 5

f = open(file, encoding='UTF-8')
fit_sum = float( 0 )
time_sum = float( 0 )
best_fit = 0
for i in range( test_runs ):
    
    # population size
    popsize = 10 
    #generations
    genlimit = 10
    #ktournament size
    kt = 5
    #initial sigma
    isigma = 10
    #mutation sigma
    msigma = 7
    #mutation chance(1/1+x)
    mchance = 2
    #crossover chance(1/1+x)
    cochance = 2
    #elitismn
    elitismn = 2

    print('-----------------------------')
    print ('TEST %i/%i' % ( i + 1, test_runs ))
    f.seek( 0 ) 
    result = run_alg( f, tmax, N )
    fit_sum += result[0]
    time_sum += result[1]
    best_fit = result[0] if result[0] > best_fit else best_fit
print('*******************************')
print("Summary:")
print(tmax, fit_sum / test_runs, time_sum / test_runs , best_fit)
f.close()